In [11]:
class Links:

    #Class variables
    #links = []
    #indices = []
    #unordered_indices = []
    #numLinks = len(unordered_indices)

    n_ts = {"tdc" : 47, "high" : 8, "medium" : 10, "low" : 12}    # Creating a dictionary with sl and the corresponding number of time-slots in the channel
    channel_ts = {1 : 8, 2 : 10, 3 : 12}
    channel = {"tdc" : 0, "high" : 1, "medium" : 2, "low" : 3}    # A dictionary for channel from sl 
    priority = {0 : "tdc", 1 : "high", 2 : "medium", 3 : "low"}    # A dictionary for sl from channel
    
    total_ts = channel_ts[1] + channel_ts[2] + channel_ts[3] 
    
    def __init__(self, nodes, weight):    # nodes = (s, d); is a tuple so as to not confuse s and d individually
        self.nodes = nodes
        self.weight = weight
        
        self.lambda_tdc = np.ones(47, dtype = bool)
        
        #self.total_ts = Links.total_ts
        
        self.occupied_ts = np.zeros(4).astype(int)    # Stores the number of occupied time slots [total, q1, q2, q3]
        self.available_ts = np.array([Links.total_ts,    # Stores the number of available time slots [total, q1, q2, q3]
                                     Links.channel_ts[1], Links.channel_ts[2], Links.channel_ts[3]])
        
        self.lambda_q1 = np.ones(Links.channel_ts[1], dtype = bool)    # for high sl
        self.lambda_q2 = np.ones(Links.channel_ts[2], dtype = bool)    # for medium sl    
        self.lambda_q3 = np.ones(Links.channel_ts[3], dtype = bool)    # for low sl

   
    def update_link(self, channel, slot):
        # call : links[index].update_link(channel, ts); where ts is timeslot
        #QSC = ["lambda_q1", "lambda_q2", "lambda_q3"]
        QSC = [1, 2, 3]
        TDC = [0]
        AC = QSC + TDC
        #if channel not in AC or ts < 0:
            #print("Invalid Update request!")

        if channel == 0:
            self.lambda_tdc[slot] = False
            #print(f"The updated tdc resources for {self.nodes} are : {self.lambda_tdc}")
        
        elif channel == 1 and slot < len(self.lambda_q1):
            self.lambda_q1[slot] = False 
            print(f"The updated q1 resources for {self.nodes} are : {self.lambda_q1}")
            
        elif channel == 2 and slot < len(self.lambda_q2):
            self.lambda_q2[slot] = False
            print(f"The updated q2 resources for {self.nodes} are : {self.lambda_q2}")
            
        elif channel == 3 and slot < len(self.lambda_q3):
            self.lambda_q3[slot] = False
            print(f"The updated q3 resources for {self.nodes} are : {self.lambda_q3}")

        else:
            print("Invalid time slot value")

        if channel != 0:
            self.occupied_ts[channel] += 1 
            self.occupied_ts[0] += 1

            self.available_ts[channel] -= 1 
            self.available_ts[0] -= 1
            

    # this can be replaced by "link.occupied_ts[channel]" and "link.available_ts[channel]"
    def ts_count(self, channel, mode = "utilized"):
        
        if channel == 1:
            count = np.count_nonzero(self.lambda_q1)    # zero => utilized slots
        elif channel == 2:
            count = np.count_nonzero(self.lambda_q2) 
        elif channel == 3:
            count = np.count_nonzero(self.lambda_q3) 

        if mode == "available":
            return count
            
        utilized_count = Links.channel_ts[channel] - count
        return utilized_count
        
    
    def display_info(self, wl_info = False):
        q1_count = self.available_ts[1] # nonzero => Available slots
        q2_count = self.available_ts[2] #== True
        q3_count = self.available_ts[3] #== True
        '''
        q1_count = np.count_nonzero(self.lambda_q1) # nonzero => Available slots
        q2_count = np.count_nonzero(self.lambda_q2) #== True
        q3_count = np.count_nonzero(self.lambda_q3) #== True
        '''
        tdc_count = np.count_nonzero(self.lambda_tdc) #== True
        print(f"Link {self.nodes} : lambda_tdc_count = {tdc_count}, lambda_q1_count = {q1_count}, lambda_q2_count = {q2_count}, lambda_q3_count = {q3_count}, occupied_ts = {self.occupied_ts}, available_ts = {self.available_ts}")

        if wl_info:    # To show the wavelength occupancy
            print(f"QSC: \n q1 : {self.lambda_q1}, q2 : {self.lambda_q2}, q3 : {self.lambda_q3}, tdc : {self.lambda_tdc}")
        
  ###################################################################################################################################################
    
    @classmethod
    def initialize_links(cls, edges):    # Initializing all the links and the individual link resources

        all_values = [nodes for row in edges for nodes in row[:2]]    # A list of all the values/nodes in first 2 columns : s & d
        m = min(all_values)
        M = max(all_values)
        numNodes = max(all_values) - min(all_values) + 1     # Correction for the convention in indexing of nodes in the data of edges
    
        links = np.zeros([M+1, M+1], dtype = object)    # Will have redundant entries. M is the highest node index
        #link_indices = np.zeros_like(links, dtype = bool)

        #links = {}
        unordered_indices = []
        for (s, d, w) in edges:

            nodes = (s, d)
            link = cls(nodes, w)
            # In the above call of __init__ constructor, the wavelength resources have also been initialized to all available(True)
    
            unordered_indices.append(nodes)
            
            # Storing in the links array
            links[s, d] = link
            links[d, s] = link
                    
        indices = links != 0    # A mask that holds the location of each link. NOTE : It stores ordered pair

        cls.links = links
        cls.indices = indices
        cls.unordered_indices = unordered_indices
        
        return links, indices, unordered_indices
        

    @classmethod
    def path_resources(cls, path, sl):    # To check for the availablr time slots, following the continuity constraint
        available_tdcs = [True] * cls.n_ts["tdc"]    # For traditional data channel slots
        available_ts = [True] * cls.n_ts[sl]    # Creating a base boolean array of the size corresponding to the specific CR's sl

        #print("available_ts: ", available_ts)
        
        for s, d in zip(path, path[1:]):    # Taking consecutive pairs of nodes and selecting the particular channel
            if sl == "high":
                band = cls.links[s, d].lambda_q1
                #print(f"link : {s, d}, lambda_q1 : {band}")
            elif sl == "medium":
                band = cls.links[s, d].lambda_q2
                #print(f" link : {s, d}, lambda_q2 : {band}")
            else:
                band = cls.links[s, d].lambda_q3
                #print(f"link : {s, d}, lambda_q3 : {band}")

            # Checking for continuity constraints in ts of quantum channel
            available_ts = [a and b for a, b in zip(available_ts, band)]    
            # print(f" link : {s, d}, available_ts : {available_ts}")
            available_tdcs = [a and b for a, b in zip(available_tdcs, links[s, d].lambda_tdc)]
            #print(f" link : {s, d}, available_tdcs : {available_tdcs}")
            
        return available_ts, available_tdcs

        
    @classmethod
    def ASSL(cls, NT, OT, n, beta_2 = 0.5):
        
        '''
        n = channel[sl]    # Could be 1 or 2 
            
        NT = []
        OT = []
        for i in range(1, 4):
            nt = n_ts[cls.priority[i]]    # Total time slot in channel/n_th wavelength
            NT.append(nt)
                
            available_ts, available_tdcs = cls.path_resources(path, priority[i])    # Available ts in the path for a particular wavelength
            ot = nt - available_ts    # Denotes the number of time-slot continuity occupied along the path
            OT.append(ot)
                
        sl = ASLC(NT, OT, n)    #ASLC(list, list, channel) 
        '''
        
        '''
        OTn denotes the number of time slots, which has been already occupied in the nth wavelength, NTn denotes all the time slots in the nth wavelength,
        and QWi denotes the wavelength position occupied by the ith CR. By mediating the parameters, 1 and 2, the wavelength and the time-slot resource 
        occupation can be changed.
        '''
        # Lower the index of the channel by 1, since I've assigned indexing from 1
        n -= 1    # here n could be either 0 or 1
        
        # Reverse the lists NT and OT, because in the paper they've taken lower number to be of lower priority
        #OT = OT[::-1]
        #NT = NT[::-1]
        
        if OT[n] >= beta_2 * NT[n]:
            QW = cls.priority[n+1]
        else:
            QW = priority[n]

        return QW
        
    @classmethod
    def FF(cls, path, cr):
        # First check for availability of resources
        # We've not considered ASLC and Resource usage extent here]

        sl = cr.sl

        '''
        if sl != "high":
            n = channel[sl]    # Could be 1 or 2 
            
            NT = []
            OT = []
            for i in range(1, 4):
                nt = n_ts[cls.priority[i]]    # Total time slot in channel/n_th wavelength
                NT.append(nt)
                
                available_ts, available_tdcs = cls.path_resources(path, priority[i])    # Available ts in the path for a particular wavelength
                ot = nt - available_ts    # Denotes the number of time-slot continuity occupied along the path
                OT.append(ot)
                
            sl = ASSL(NT, OT, n)    #ASLC(list, list, channel)  
         '''   
        
        available_ts, available_tdcs = cls.path_resources(path, sl)    # List of available time slots in the channel along the entire path

        
        if cr.tk > len(available_ts):    # ts is 1 in our case by default. But in case we change it, this line will be useful
            raise ValueError("CR duration cannot be longer than the available time-slots in a channel")

        print("np.count_nonzero(available_ts) :", np.count_nonzero(available_ts))
         
        if np.count_nonzero(available_ts) == 0 or np.count_nonzero(available_tdcs) == 0:    # A pre-condition to check if available slots are present
            cr.update_status("blocked")
            print(f"CR {cr.index} blocked on the current path")
            return False

        # Some issue here
        qscs = np.nonzero(available_ts)[0][0]    # This function will give the index of 1st non-zero/True element in the list
        print("qscs :", qscs)
        tdcs = np.nonzero(available_tdcs)[0][0]
        
        for s, d in zip(path, path[1:]):    # A loop to update all the links in the path 
            link = links[s, d]

            link.update_link(cls.channel["tdc"], tdcs)
            link.update_link(cls.channel[sl], qscs)    # Updating the i_th ts in channel for sl to False

        allocated_resources = [cls.channel[sl], qscs]
        cr.update_status("allocated", allocated_resources, path)
        print(f"allocated ts_{qscs} successfully to CR {cr.index}")
        return True
        """
        for i, ts in enumerate(available_ts) :
            if ts:    # If a time slot is available in the required channel of QSC
                for s, d in zip(path, path[1:]):    # A loop to update all the links in the path 
                    link = links[s, d]

                    # For traditional wavelength allocation
                    '''
                    for j, slot in enumerate(available_tdcs) :    # Fetching a available slot on tdc
                        if slot :
                            link.update_link(cls.channel["tdc"], j)
                            link.update_link(cls.channel[sl], i)    # Updating the i_th ts in channel for sl to False
                            
                    '''
                    tdc = -1
                    for j, slot in enumerate(available_tdcs) :    # Fetching a available slot on tdc
                        if slot :
                            tdc = j    # Not allocating here because need to check for qsc as well
                            break
                    if tdc == -1:    # If no tdc slot available
                        cr.update_status("blocked")
                        return False
                        
                    #link.lambda_tdc[tdc] = False    # Reserving the tdc slot        
                    link.update_link(cls.channel["tdc"], tdc)
                    link.update_link(cls.channel[sl], i)    # Updating the i_th ts in channel for sl to False
 
                cr.update_status("allocated")
                print(f"allocated ts {i} successfully to CR {cr.index}")
                    
                return True
        """

    @classmethod
    def TUR(cls, channel = 'total'):    # Returns the total TUR, if no 2nd argument given    
        # Only for quantum channel
        
        util_ts = [0, 0, 0, 0]
        tur = util_ts
        
        link_ts = sum(cls.n_ts[priority] for priority in ["high", "medium", "low"])    # ts in each link. In the current case, it's 8+10+12 = 30
        print("Time slots in the entire QSC of each link : ", link_ts)
        
        print("Number of links : ", len(cls.links)) 
        
        total_ts = len(cls.links) * link_ts
        print("Total number of QSC time slots in the network : ", total_ts)
        
        for nodes in cls.unordered_indices :
            link = cls.links[nodes]
            
            for i in range(3):
                util_ts[i+1] += link.ts_count(i+1)

        util_ts[0] = sum(util_ts[i+1] for i in range(3))
        print("The number of utilized time-slots : ", util_ts)

        tur[0] = util_ts[0]/total_ts
        for i in range(3):
            tur[i+1] = util_ts[i+1]/(15 * (8 + i*2))
            #tur[i+1] = util_ts[i+1]/(len(cls.links)*cls.channel_ts[i+1])
        print("\n The time-slot utilization ratio(TUR) is : ", tur)
        
        if channel == 'all':
            return tur

        else: 
            return tur[0]       
    
    @classmethod
    def display_all_links(cls, wl_info = False):
        for nodes in cls.unordered_indices :
            cls.links[nodes].display_info(wl_info)


expected output [False, True, False, False, True, True, False, True]